# Predictive Asset Maintenance with XGBoost

From the [Sisyphean Gridworks ML Playground](https://sgridworks.com/ml-playground/guides/04-predictive-maintenance.html)

## Setup

Clone the repository and install dependencies. Run this cell first.

In [ ]:
!git clone https://github.com/SGridworks/Dynamic-Network-Model.git 2>/dev/null || echo 'Already cloned'
%cd Dynamic-Network-Model
!pip install -q pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm pyarrow

## Load the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

from demo_data.load_demo_data import (
    load_transformers, load_outage_history, load_weather_data
)

transformers = load_transformers()
outages      = load_outage_history()

print(f"Transformers:  {len(transformers)}")
print(f"Outage events: {len(outages)}")

## Explore the Transformer Data

In [ ]:
# What columns do we have?
print(transformers.columns.tolist())
print(transformers.head())

# Distribution of transformer age
plt.figure(figsize=(8, 4))
plt.hist(transformers["age_years"], bins=20, color="#5FCCDB", edgecolor="white")
plt.title("Transformer Age Distribution")
plt.xlabel("Age (years)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

## Create the Failure Target

We need to label each transformer: has it experienced an equipment-failure outage? We'll use the outage history to identify feeders linked to "equipment failure" causes and flag the transformers on those feeders.

In [ ]:
# Rated kVA distribution
plt.figure(figsize=(8, 4))
plt.hist(transformers["kva_rating"], bins=20, color="#2D6A7A", edgecolor="white")
plt.title("Transformer Rated kVA Distribution")
plt.xlabel("Rated kVA")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

## Engineer Maintenance Features

Outage history can serve as a proxy for maintenance exposure. Feeders with frequent or long-duration outages suggest areas where equipment is under greater stress.

In [ ]:
# Filter outages to equipment failures only
equip_failures = outages[outages["cause_code"] == "equipment failure"]

# Count equipment-failure outages per feeder
failure_counts = equip_failures.groupby("feeder_id").size().reset_index(
    name="failure_count"
)

# Merge with transformer table on feeder_id
df = transformers.merge(failure_counts, on="feeder_id", how="left")
df["failure_count"] = df["failure_count"].fillna(0).astype(int)

# Binary target: has this transformer's feeder had equipment failures?
df["has_failed"] = (df["failure_count"] > 0).astype(int)

print(f"Transformers with failures:    {df['has_failed'].sum()}")
print(f"Transformers without failures: {(df['has_failed'] == 0).sum()}")

## Prepare Features and Split

In [ ]:
# Count all outage events per feeder
outage_stats = outages.groupby("feeder_id").agg(
    total_outages=("fault_detected", "count"),
    avg_outage_duration=("duration_hours", "mean")
).reset_index()

# Merge into main table
df = df.merge(outage_stats, on="feeder_id", how="left")

# Fill feeders with no outage records
df["total_outages"] = df["total_outages"].fillna(0)
df["avg_outage_duration"] = df["avg_outage_duration"].fillna(0)

print(df[["feeder_id", "age_years", "kva_rating",
          "total_outages", "has_failed"]].head(10))

## Train the XGBoost Model

In [ ]:
# Define features
feature_cols = [
    "age_years", "kva_rating",
    "total_outages", "avg_outage_duration"
]

X = df[feature_cols]
y = df["has_failed"]

# Split 70/30 (smaller dataset so we keep more for testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples:     {len(X_test)}")

## Test and Evaluate

In [ ]:
# Calculate class imbalance ratio for XGBoost
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

model = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    scale_pos_weight=neg / pos,  # handle class imbalance
    random_state=42,
    eval_metric="logloss"
)

model.fit(X_train, y_train)
print("XGBoost training complete.")

## Plot the ROC Curve

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # probability of failure

# Classification report
print(classification_report(y_test, y_pred,
      target_names=["No Failure", "Failure"]))

# AUC score
auc = roc_auc_score(y_test, y_prob)
print(f"AUC-ROC Score: {auc:.3f}")

## Generate a Risk-Ranked Asset List

The real value of this model is not just accuracy—it's the ability to produce a prioritized list of assets that maintenance crews can act on.

In [ ]:
# ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(fpr, tpr, color="#5FCCDB", linewidth=2, label=f"XGBoost (AUC = {auc:.3f})")
ax.plot([0, 1], [0, 1], color="gray", linestyle="--", label="Random Guess")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curve: Transformer Failure Prediction")
ax.legend()
plt.tight_layout()
plt.show()

## Feature Importance

In [ ]:
# Score every transformer (not just the test set)
df["failure_risk_score"] = model.predict_proba(df[feature_cols])[:, 1]

# Sort by risk (highest first)
risk_list = df.sort_values("failure_risk_score", ascending=False)

print("Top 10 Highest-Risk Transformers:")
print(risk_list[["feeder_id", "age_years", "kva_rating",
                "failure_risk_score"]].head(10).to_string(index=False))

## What You Built and Next Steps

In [ ]:
# Which factors contribute most to failure risk?
importances = pd.Series(model.feature_importances_, index=feature_cols)
importances = importances.sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(8, 5))
importances.plot(kind="barh", color="#5FCCDB", ax=ax)
ax.set_title("Feature Importance: What Drives Transformer Failure?")
ax.set_xlabel("Importance Score")
plt.tight_layout()
plt.show()